In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceshiptitanic/sample_submission.csv
/kaggle/input/spaceshiptitanic/train.csv
/kaggle/input/spaceshiptitanic/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/spaceshiptitanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceshiptitanic/test.csv')

In [3]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
train_data.corr()['Transported']

Age            -0.075026
RoomService    -0.244611
FoodCourt       0.046566
ShoppingMall    0.010141
Spa            -0.221131
VRDeck         -0.207075
Transported     1.000000
Name: Transported, dtype: float64

In [5]:
train_data['Cabin'].nunique()

6560

In [6]:
features = ['HomePlanet','CryoSleep','Destination','VIP']
train_data = train_data.dropna(subset=['Age'])
X = pd.get_dummies(train_data[features])
X = pd.concat([X,train_data['Age']],axis=1)
y = train_data['Transported']
data = pd.concat([X,y],axis=1)
data.corr()['Transported']

HomePlanet_Earth            -0.168240
HomePlanet_Europa            0.176399
HomePlanet_Mars              0.019765
CryoSleep_False             -0.450897
CryoSleep_True               0.459296
Destination_55 Cancri e      0.108094
Destination_PSO J318.5-22    0.000124
Destination_TRAPPIST-1e     -0.093429
VIP_False                    0.024502
VIP_True                    -0.036973
Age                         -0.075026
Transported                  1.000000
Name: Transported, dtype: float64

In [7]:
x_test = pd.concat([pd.get_dummies(test_data[features]),test_data['Age']],axis=1)
x_test.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age
0,1,0,0,0,1,0,0,1,1,0,27.0
1,1,0,0,1,0,0,0,1,1,0,19.0
2,0,1,0,0,1,1,0,0,1,0,31.0
3,0,1,0,1,0,0,0,1,1,0,38.0
4,1,0,0,1,0,0,0,1,1,0,20.0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20, random_state=101)

In [9]:
X_train.shape

(6811, 11)

In [10]:
from xgboost import XGBRegressor


my_model = XGBRegressor(n_estimators=10000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=20, 
             eval_set=[(X_test, y_test)],
             verbose=False)
pred = my_model.predict(X_test)>0.5
pred

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


array([False,  True, False, ..., False,  True, False])

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.7475044039929536

In [12]:
"""
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Dropout
model = Sequential()
model.add(Dense(100,activation='sigmoid'))
model.add(Dense(25,activation='sigmoid'))
model.add(Dense(6,activation='sigmoid'))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
model.fit(x=X_train,y=y_train,epochs=200,validation_data=(X_test, y_test),callbacks=(early_stop))
"""

"\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Activation,Dense,Dropout\nmodel = Sequential()\nmodel.add(Dense(100,activation='sigmoid'))\nmodel.add(Dense(25,activation='sigmoid'))\nmodel.add(Dense(6,activation='sigmoid'))\n\nmodel.add(Dense(1,activation='sigmoid'))\nmodel.compile(loss='binary_crossentropy', optimizer='adam')\nfrom tensorflow.keras.callbacks import EarlyStopping\nearly_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)\nmodel.fit(x=X_train,y=y_train,epochs=200,validation_data=(X_test, y_test),callbacks=(early_stop))\n"

In [13]:
"""
import matplotlib.pyplot as plt
import seaborn as sns
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()
#losses[['loss']].plot()
"""

"\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nlosses = pd.DataFrame(model.history.history)\nlosses[['loss','val_loss']].plot()\n#losses[['loss']].plot()\n"

In [14]:
predictions = my_model.predict(x_test)>0.5

#predictions = predictions.reshape(predictions.shape[0]*predictions.shape[1])
predictions


array([ True, False,  True, ...,  True,  True, False])

In [15]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
